In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("NASA Exoplanet Data/NASA Exoplanet Archive - Planetary Systems Data.csv")

C:\Users\jarne\AppData\Local\Temp\ipykernel_29820\3066643026.py:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("NASA Exoplanet Data/NASA Exoplanet Archive - Planetary Systems Data.csv")


In [5]:
columns_to_select = ['rowid', 'pl_name', 'st_teff','st_lum','sy_dist','pl_orbsmax','pl_orbeccen','pl_orbper','pl_masse','pl_rade','pl_dens','pl_eqt']
selected_columns = df[columns_to_select]

print(selected_columns)

       rowid    pl_name  st_teff  st_lum   sy_dist  pl_orbsmax  pl_orbeccen  \
0          1   11 Com b      NaN     NaN   93.1846       1.210          NaN   
1          2   11 Com b   4742.0   2.243   93.1846       1.290        0.231   
2          3   11 Com b   4874.0   1.978   93.1846       1.178        0.238   
3          4   11 UMi b   4213.0     NaN  125.3210       1.530        0.080   
4          5   11 UMi b      NaN     NaN  125.3210       1.510          NaN   
...      ...        ...      ...     ...       ...         ...          ...   
36457  36458  ups And d      NaN     NaN   13.4054       2.570        0.269   
36458  36459  ups Leo b   4836.0   1.800   52.5973       1.180        0.320   
36459  36460   xi Aql b      NaN     NaN   56.1858       0.580          NaN   
36460  36461   xi Aql b   4780.0   1.839   56.1858       0.680        0.000   
36461  36462   xi Aql b   4841.0   1.767   56.1858       0.625        0.059   

        pl_orbper  pl_masse  pl_rade  pl_dens  pl_e

In [ ]:
columns_to_select = ['pl_masse','pl_rade','pl_dens','pl_eqt']
nan = df[columns_to_select]

print(nan)